<h1>Ambulance Path Optimization and Analysis of Road efficiency in Ambulance Services using Network Analysis</h1>


# Abstract

<p>The graph structure can be used to represent many real – world networks and one among them is the road network. The way how different roads are connected to each other and how different places are connected by roads can be visualized with the help of graphs. By understanding the network of roads, we can find many useful insights like shortest path decision. The shortest path decision is inevitable in sectors like delivery services, public transport services and ambulance service. The effectiveness of the ambulance resides in how fast it reaches the nearby hospital from the starting place. This project aims to find the shortest path for an ambulance in the given road network. Also, the project proposes a model that helps to analyze and report on the efficiency of any given road network in ambulance services.</p>

# Loading the required libraries

In [ ]:
import pandas as pd
import networkx as nx
import random
import matplotlib.pyplot as plt
%matplotlib inline
import warnings

from sklearn.preprocessing import LabelEncoder

# Reading the edge list as csv file and making a dataframe out of it

In [ ]:
edge_data = pd.read_csv("/kaggle/input/edge-data/edge_data.csv")
edge_data.columns = ['From','To']
edge_data.head()

# In case edge list is categorical values then,

In [ ]:
edge_data = edge_data.apply(LabelEncoder().fit_transform)
edge_data.head()

# Converting edge list dataframe into graph

In [ ]:
edges = []
for a,b,c in edge_data.to_records().tolist():
    edges.append((b,c))

graph = nx.Graph(edges)

'''Storing the total number of ndoes and edges in the graph'''
n = len(graph.nodes())
e = len(graph.edges())

print(f'\n----GRAPH INFO----\n')
print(f'Node count : {n}\n')
print(f'Edge count : {e}\n')

# Visualising the formed graph

In [ ]:
fig = plt.figure(figsize = (10,7))
nx.draw_networkx(graph, with_labels = True,node_color = 'c', node_size = 600, alpha = 0.9, edge_color = 'm')
plt.show()

In [ ]:
plt.savefig('graph.png')

# Splitting the nodes in network as *Hospital* , *Accident Prone Zone* , *Normal Places*

<h3>random.seed() function is used to ensure the same set of nodes are generated at each execution</h3>

In [ ]:
'''List to store all nodes except the hospital nodes'''
rem = []

'''Taking 1/5 nodes as hospitals'''
hosc = int(n / 5)
hos = []
el = n - 1
random.seed(3)
for i in range(0,hosc):
    hos.append(random.randint(0, el))


'''Taking 2/5 nodes as accident prone zones'''
ambuc = int(2*n/5)
ambu = []
random.seed(4)
for i in range(0,ambuc):
    r = random.randint(0,el)
    while r in hos or r in ambu:
        r = random.randint(0,el)
    ambu.append(r)
    rem.append(r)


'''Taking the remaining nodes as normal places'''
norm = []
for i in range(0,n):
    if i not in hos and i not in ambu:
        norm.append(i)
        rem.append(i)

# After splitting

In [ ]:
print('\n-----HOSPITALS-----\n')
c = 0
for i in hos:
    c += 1
    if c%20 == 0:
        print(f'{i}')
        continue
    print(f'{i}', end = ' , ') 

In [ ]:
print('\n-----ACCIDENT PRONE ZONES-----\n')
c = 0
for i in ambu:
    c += 1
    if c%20 == 0:
        print(f'{i}')
        continue
    print(f'{i}', end = ' , ') 

In [ ]:
print('\n-----NORMAL PLACES-----\n')
c = 0
for i in norm:
    c += 1
    if c%20 == 0:
        print(f'{i}')
        continue
    print(f'{i}', end = ' , ') 

# Finding out the degree distribution

In [ ]:
noddegs = nx.degree(graph)

print('\n-----DEGREE DISTRIBUTION-----\n')
c = 0
for i in range(0,n):
    c += 1
    if c%4 == 0:
        print(f'Node {i} degree : {noddegs[i]}')
        continue
    print(f'Node {i} degree : {noddegs[i]}', end = '\t')

# Finding degree centrality of all nodes

In [ ]:
noddegcen = nx.degree_centrality(graph)

print('\n-----DEGREE CENTRALITY DISTRIBUTION-----\n')
c = 0
for i in range(0,n):
    c += 1
    if c%4 == 0:
        print(f'Node {i} : {noddegcen[i]:.3f}')
        continue
    print(f'Node {i} : {noddegcen[i]:.3f}', end = '\t')

# Finding node with maximum degree centrality

In [ ]:
maxdegcen = max(noddegcen.values())
maxcenn = 0
for i in noddegcen.keys():
    if maxdegcen == noddegcen[i]:
        maxcenn = i
        break
print(f'\nMaximum Degree Centrality Node : {maxcenn}\n')

# Finding eigen vector centrality of all nodes

In [ ]:
nodeig = nx.eigenvector_centrality(graph, max_iter = 1000)
print('\n-----EIGEN CENTRALITY DISTRIBUTION-----\n')
c = 0
for i in range(0,n):
    c += 1
    if c%4 == 0:
        print(f'Node {i} : {nodeig[i]:.3f}')
        continue
    print(f'Node {i} : {nodeig[i]:.3f}', end = '\t')

# Finding node with maximum eigen vector centrality

In [ ]:
maxeig = max(nodeig.values())
maxeign = 0
for i in nodeig.keys():
    if maxeig == nodeig[i] and i in norm:
        maxeign = i
        break
print(f'\nMaximum Eigen Vector Centrality Node : {maxeign}\n')

# For Every *Accident Prone Zone* printing the shortest path to *Hospitals*, if it exists

In [ ]:
def accshphos(graph):
    print('\n---AMBULANCE ROUTE FROM EVERY ACCIDENT PRONE ZONE---\n')
    thc = 0
    nambu = 0
    npambu = 0
    for i in ambu:
        hc = 0
        mhc = 0
        print(f'\n---FOR ACCIDENT PRONE ZONE {i}---\n')
        for j in hos:
            if nx.has_path(graph,i,j):
                l = nx.shortest_path(graph,i,j)
                sl = l[:-1]
                if hos in sl:
                    mhc += 1
                hc += 1
                thc += 1
                print(f'\nRoute To Hospital {j} : {l}')
        print(f'\nHospitals Connected : {hc}')
        print(f'\nHospitals Disconnected :{hosc - hc}')
        print(f'\nRoutes with connecting more than one hospital : {mhc}\n')
        if hc == hosc:
            nambu+=1
        if hc == 0:
            npambu += 1
    print('\n----------------------------------------------------\n')

    avghc = int(thc / ambuc)
    print(f'\nMean of hospitals connected : {avghc}\n')
    if avghc == hosc:
        print('\nEvery accident prone zone is connected to every other hospital\n')
    
    return (nambu,npambu)

# Function to calculate the *Hospital Distance Dictionary*

In [ ]:
'''

Hospital Distance Dictionary 
    1. The keys are the all nodes in graphs except the nodes labelled  as 'Hospitals'.
    2. The values for these keys are list, which represents the length of the shortest path from the key node to each hospital
       node in the graph.
       
'''
dis = {}
def dismat(graph):
    tssp = 0
    for i in rem:
        row = []
        for h in range(len(hos)):
            l = 0
            
            '''Checking if there is shortest path between given (node, hospital node)'''
            
            '''If True adding the shortest path length'''
            if nx.has_path(graph,i,hos[h]):
                l = nx.shortest_path_length(graph,i,hos[h])            
            else:
                l = -1
            row.append(l)  
        dis[i] = row
    
    print(f'\n-----HOSPITAL DISTANCE DICTIONARY-----\n')
    for i in dis.keys():
        print(f'\n----FOR NODE {i}----\n')
        for h in range(len(hos)):
            print(f'Distance from Hospital {hos[h]} : {dis[i][h]}\n')
            tssp += dis[i][h]
        print(f'\n##################################################\n')
        
    return tssp

# Function to find the choice list of nodes for a given node

In [ ]:
'''

The choice list is done based on two parameters
    1. Number of hospitals connected through that node.
    2. Sum of shortest path lengths from that node to all the hospitals nodes connected to it.

The preference of parameters is
    1. The nodes with more count of hospitals connected will come first.
    2. If two nodes connect to same number of hospitals then, the node with minimum sum of shortest path lenghts comes first.
    
'''
def nxtnodli(graph,node):
    nei = []
    for ni in nx.all_neighbors(graph,node):
        if ni not in hos:
            nei.append(ni)
    sspl = []
    
    for i in nei:
        ssp = 0
        nhc = 0
        for h in range(len(hos)):
            if dis[i][h] != -1:
                nhc += 1
                ssp += dis[i][h]
        lab = ''
        if i in ambu:
            lab = 'Accident Prone Zone'
        else:
            lab = 'Normal Place'
        sspl.append((i,nhc,ssp,lab))
    
    '''First sorting is done based on number of hospitals nodes connected'''
    snei = sorted(sspl, key = lambda k : k[1], reverse = True)
    
    '''Second sorting is done based on the sum of shortest path lengths'''
    snei = sorted(snei, key = lambda k : k[2])
    
    return snei

# Finding the fortunate *Accident Prone Zone*

In [ ]:
def lucacc(graph):
    lucacc = []
    for i in ambu:
        ssp = 0
        nh = 0
        for h in range(len(hos)):
            if dis[i][h] != -1:
                ssp += dis[i][h]
                nh += 1
        lucacc.append((i,nh, ssp))
                      
    lucacc = sorted(lucacc, key = lambda key : key[2])
    lucacc = sorted(lucacc, key = lambda key : key[1], reverse = True)
    
    print(f'\n-----FORTUNATE ACCIDENT PRONE ZONE-----\n')

    for a,b,c in lucacc:
        print(f'\nACCIDENT PRONE ZONE : {a}\n')
        print(f'TOTAL HOSPITALS CONNECTED : {b}\n')
        print(f'SUM OF SHORTEST PATHS TO CONNECTED HOSPITALS : {c}\n')
        print(f'################################################\n')
    return          

# Finding the *Busy Road*

In [ ]:
def busrod(graph):
    edges = graph.edges()
    
    
    ambushos = {}
    road = []
    
    for i in ambu:
        for h in hos:
            if nx.has_path(graph,i,h):
                l = nx.shortest_path(graph,i,h)
                led  = []
                for e in range((len(l) - 1)):
                    led.append((l[e], l[e+1]))
                ambushos[(i,h)] = led
    
    for a,b in edges:
        t1 = (a,b)
        t2 = (b,a)
        occ = 0
        for j in ambushos.values():
            if t1 in j or t2 in j:
                occ += 1
        
        road.append((t1,occ))
    
    
    road = sorted(road, key = lambda k : k[1], reverse = True)
    maxocc = road[0][1]
        
    busroad = []
    for a,b in road:
        if b == maxocc:
            busroad.append((a,b))           
    return busroad

# Finding the *Most Serving Hospital*

In [ ]:
def moshos(graph):
    
    mososp = ()
    hossp = []
    for i in range(len(hos)):
        na = 0
        ssa = 0
        for j in ambu:
            if dis[j][i] != -1:
                na += 1
                ssa += dis[j][i]
        hossp.append((hos[i],na,ssa))
    
    
    hossp = sorted(hossp, key = lambda k : k[2])
    hossp = sorted(hossp, key = lambda k : k[1], reverse = True)
    
    moshoss = []
    moshosc = hossp[0]
    for i in hossp:
        if i[1] == moshosc[1] and i[2] == moshosc[2]:
            moshoss.append(i)

    
    print('\n-----MOST SERVING HOSPITAL-----\n')
    
    for a,b,c in moshoss:
        print(f'Hospital : {a}\n')
        print(f'Accident Prone Zones Connected : {b}\n')
        print(f'Sum of shortest path lengths to connected accident prone zones : {c}\n')
        print('-------------------------------------------------------------\n')
    
    return

# Creating a list to store the analysis result of each sample

In [ ]:
rec = []

# Analysis of Graph 1

In [ ]:
pambu = 0
poambu = 0
perrod = 0

In [ ]:
#Finding the shortest path between every pair of connected accident prone zone and hospital
(pambu,poambu) = accshphos(graph)

In [ ]:
#Finding the Hospital Distanc Dictionary
perrod = dismat(graph)

In [ ]:
#Finding the next node choice  list for any given node

#Input is preloaded due to unavailablity of data input at the time of saving version
s = 5

cl = []
'''Checking if hospital nodes are immediate neigbours of given node'''
for i in hos:
    if i in nx.all_neighbors(graph,s):
        cl.append((i,1,0,'Hospital'))

'''Getting the choice list from function'''
for i in nxtnodli(graph,s):
        cl.append(i)
        
print('\n-----NODES TO CHOOSE NEXT-----\n')
for a,b,c,d in cl:
    print(f'Node : {a}\n')
    print(f'Node label : {d}\n')
    print(f'Total hospitals Connected through this node : {b}\n')
    print(f'Sum of shortest path lengths to connected hospitals : {c}\n')
    print('-------------------------------------------------------------\n')

In [ ]:
# Finding the next node choice list for all accident prone zones

for pz in ambu:
    cl = []
    '''Checking if hospital nodes are immediate neigbours of given node'''
    for i in hos:
        if i in nx.all_neighbors(graph,pz):
            cl.append((i,1,0,'Hospital'))

    '''Getting the choice list from function'''
    for i in nxtnodli(graph,pz):
            cl.append(i)

    print(f'\n-----FOR ACCIDENT PRONE ZONE {pz}-----\n')
    for a,b,c,d in cl:
        print(f'Node : {a}\n')
        print(f'Node label : {d}\n')
        print(f'Total hospitals Connected through this node : {b}\n')
        print(f'Sum of shortest path lengths to connected hospitals : {c}\n')
        print('-------------------------------------------------------------\n')

In [ ]:
#Finding the Fortunate Accident Prone Zone
lucacc(graph)

In [ ]:
#Finding the Most Serving Hospital 
moshos(graph)

In [ ]:
#Finding the Most Busy road
l = busrod(graph)
print(f'\n-----MOST BUSY ROADS-----\n')
for a,b in l:
    s,t = a
    print(f'\nSource Node: {s}\n')

    sl = ''
    tl = ''

    if s in ambu:
        sl = 'Accident Prone Zone'
    elif s in hos:
        sl = 'Hospital'
    else:
        sl = 'Normal Place'

        
    if t in ambu:
        tl = 'Accident Prone Zone'
    elif t in hos:
        tl = 'Hospital'
    else:
        tl = 'Normal Place'

    print(f'\nSource Node Label : {sl}\n')
    print(f'\nTarget Node : {t}\n')
    print(f'\nTarget Node Lablel : {tl}\n')
    print(f'\nNumber OF Routes Connects : {b}\n')
    print(f'\n###################################\n')

In [ ]:
rec.append(('Graph',e,pambu,poambu, perrod))

# Removing the *Busy Road* and observing the change in analysis

In [ ]:
graphbsr = nx.Graph(edges)
l = busrod(graphbsr)
for a,b in l:
    s,t = a
    graphbsr.remove_edge(s,t)

In [ ]:
'''Storing the total number of ndoes and edges in the graph'''
brn = len(graphbsr.nodes())
bre = len(graphbsr.edges())

print(f'\n----GRAPH INFO----\n')
print(f'Node count : {brn}\n')
print(f'Edge count : {bre}\n')

# Visualising the new graph formed

In [ ]:
fig = plt.figure(figsize = (10,10))
nx.draw_networkx(graphbsr, with_labels = True,node_color = 'c', node_size = 600, alpha = 0.9, edge_color = 'm')
plt.show()

# Analysis of Graph 2

In [ ]:
brpambu = 0
brpoambu = 0
brperrod = 0

In [ ]:
#Finding the shortest path between every pair of connected accident prone zone and hospital
(brpambu,brpoambu) = accshphos(graphbsr)

In [ ]:
#Finding the Hospital Distanc Dictionary
brperrod = dismat(graphbsr)

In [ ]:
##### Finding the next node choice  list for any given node

#Input is preloaded due to unavailablity of data input at the time of saving version
s = 3

cl = []
'''Checking if hospital nodes are immediate neigbours of given node'''
for i in hos:
    if i in nx.all_neighbors(graphbsr,s):
        cl.append((i,1,0,'Hospital'))

'''Getting the choice list from function'''
for i in nxtnodli(graphbsr,s):
        cl.append(i)
        
print('\n-----NODES TO CHOOSE NEXT-----\n')
for a,b,c,d in cl:
    print(f'Node : {a}\n')
    print(f'Node label : {d}\n')
    print(f'Total hospitals Connected through this node : {b}\n')
    print(f'Sum of shortest path lengths to connected hospitals : {c}\n')
    print('-------------------------------------------------------------\n')

In [ ]:
# Finding the next node choice list for all accident prone zones

for pz in ambu:
    cl = []
    '''Checking if hospital nodes are immediate neigbours of given node'''
    for i in hos:
        if i in nx.all_neighbors(graphbsr,pz):
            cl.append((i,1,0,'Hospital'))

    '''Getting the choice list from function'''
    for i in nxtnodli(graphbsr,pz):
            cl.append(i)

    print(f'\n-----FOR ACCIDENT PRONE ZONE {pz}-----\n')
    for a,b,c,d in cl:
        print(f'Node : {a}\n')
        print(f'Node label : {d}\n')
        print(f'Total hospitals Connected through this node : {b}\n')
        print(f'Sum of shortest path lengths to connected hospitals : {c}\n')
        print('-------------------------------------------------------------\n')

In [ ]:
#Finding the Fortunate Accident Prone Zone
lucacc(graphbsr)

In [ ]:
#Finding the Most Serving Hospital
moshos(graphbsr)

In [ ]:
#Finding the Most Busy Road
l = busrod(graphbsr)
print(f'\n-----MOST BUSY ROADS-----\n')
for a,b in l:
    s,t = a
    print(f'\nSource Node: {s}\n')

    sl = ''
    tl = ''

    if s in ambu:
        sl = 'Accident Prone Zone'
    elif s in hos:
        sl = 'Hospital'
    else:
        sl = 'Normal Place'

        
    if t in ambu:
        tl = 'Accident Prone Zone'
    elif t in hos:
        tl = 'Hospital'
    else:
        tl = 'Normal Place'

    print(f'\nSource Node Label : {sl}\n')
    print(f'\nTarget Node : {t}\n')
    print(f'\nTarget Node Lablel : {tl}\n')
    print(f'\nNumber OF Routes Connects : {b}\n')
    print(f'\n###################################\n')

In [ ]:
rec.append(('Graph with busy road removed',bre,brpambu, brpoambu, brperrod))

# Removing *2/5* edges and observing the change in analysis

In [ ]:
grapher = nx.Graph(edges)
c = int(0.4*e)
for i in range(c):
    s = random.choice(list(grapher.nodes()))
    t = random.choice(list(grapher.nodes()))
    while not grapher.has_edge(s,t):
        s = random.choice(list(grapher.nodes()))
        t = random.choice(list(grapher.nodes()))
    print(f'--- REMOVED EDGE : {s} -> {t} ---\n')
    grapher.remove_edge(s,t)

In [ ]:
'''Storing the total number of ndoes and edges in the graph'''
ern = len(grapher.nodes())
ere = len(grapher.edges())

print(f'\n----GRAPH INFO----\n')
print(f'Node count : {ern}\n')
print(f'Edge count : {ere}\n')

# Visualising the new graph formed

In [ ]:
print(f'Total edges before removing : {e}\n')
print(f'Total edges after removing : {ere}')

In [ ]:
fig = plt.figure(figsize = (10,10))
nx.draw_networkx(grapher, with_labels = True,node_color = 'c', node_size = 600, alpha = 0.9, edge_color = 'm')
plt.show()

# Analysis of Graph 3

In [ ]:
erpambu = 0
erpoambu = 0
erperrod = 0

In [ ]:
#Finding the shortest path between every pair of connected accident prone zone and hospital
(erpambu,erpoambu) = accshphos(grapher)

In [ ]:
#Finding the Hospital Distanc Dictionary
erperrod = dismat(grapher)

In [ ]:
#Finding the next node choice  list for any given node

#Input is preloaded due to unavailablity of data input at the time of saving version
s = 6

cl = []
'''Checking if hospital nodes are immediate neigbours of given node'''
for i in hos:
    if i in nx.all_neighbors(grapher,s):
        cl.append((i,1,0,'Hospital'))

'''Getting the choice list from function'''
for i in nxtnodli(grapher,s):
        cl.append(i)
        
print('\n-----NODES TO CHOOSE NEXT-----\n')
for a,b,c,d in cl:
    print(f'Node : {a}\n')
    print(f'Node label : {d}\n')
    print(f'Total hospitals Connected through this node : {b}\n')
    print(f'Sum of shortest path lengths to connected hospitals : {c}\n')
    print('-------------------------------------------------------------\n')

In [ ]:
# Finding the next node choice list for all accident prone zones

for pz in ambu:
    cl = []
    '''Checking if hospital nodes are immediate neigbours of given node'''
    for i in hos:
        if i in nx.all_neighbors(grapher,pz):
            cl.append((i,1,0,'Hospital'))

    '''Getting the choice list from function'''
    for i in nxtnodli(grapher,pz):
            cl.append(i)

    print(f'\n-----FOR ACCIDENT PRONE ZONE {pz}-----\n')
    for a,b,c,d in cl:
        print(f'Node : {a}\n')
        print(f'Node label : {d}\n')
        print(f'Total hospitals Connected through this node : {b}\n')
        print(f'Sum of shortest path lengths to connected hospitals : {c}\n')
        print('-------------------------------------------------------------\n')

In [ ]:
#Finding the Fortunate Accident Prone Zone
lucacc(grapher)

In [ ]:
#Finding the Most Serving Hospital
moshos(grapher)

In [ ]:
#Finding the Most Busy Road
l = busrod(grapher)
print(f'\n-----MOST BUSY ROADS-----\n')
for a,b in l:
    s,t = a
    print(f'\nSource Node: {s}\n')

    sl = ''
    tl = ''

    if s in ambu:
        sl = 'Accident Prone Zone'
    elif s in hos:
        sl = 'Hospital'
    else:
        sl = 'Normal Place'

        
    if t in ambu:
        tl = 'Accident Prone Zone'
    elif t in hos:
        tl = 'Hospital'
    else:
        tl = 'Normal Place'

    print(f'\nSource Node Label : {sl}\n')
    print(f'\nTarget Node : {t}\n')
    print(f'\nTarget Node Lablel : {tl}\n')
    print(f'\nNumber OF Routes Connects : {b}\n')
    print(f'\n###################################\n')

In [ ]:
rec.append(('Graph with 2/5 roads removed',ere,erpambu,erpoambu, erperrod))

# Forming a data frame from results of all samples

In [ ]:
'''
Perfect Accident Prone Zone : Accident Prone Zone that is connected to every hospital present in the network
Poor Accident Prone Zone : Accident Prone Zone that is not connected to any hospital present in the network
'''
compana = pd.DataFrame(rec, columns = ['Graph','Edges','Perfect_Accident_Prone_Zone','Poor_Accident_Prone_Zone','Sum_Shortest_Paths'])

compana

<h3>PROJECT MENTOR : Dr. L. Jani Anbrasi</h3>

<h3>OTHER COLLOBORATORS</h3>
<h5>Sumegh S Gonugade</h5>
<h5>Anchana V</h5>